In [10]:
### Convection in annulus example

# +
import petsc4py
from petsc4py import PETSc

import underworld3 as uw
from underworld3.systems import Stokes
from underworld3 import function

import numpy as np

import os

import sympy

In [11]:
### set reference values
outerRadius   = 6370e3
internalRadius= (6370e3 - 660e3) ### UM - LM transition
innerRadius   = 3480e3
refLength     = (outerRadius - innerRadius) ### thickness of mantle

rI   = innerRadius / refLength
rInt = internalRadius / refLength
rO   = outerRadius / refLength

# +
mu    = 1.
kappa = 1.
alpha = 1.

rho   = 1.

T_cmb = 1.
T_surf= 0.

Ra_number = 1e6

res = 0.075

In [12]:
### FS - free slip top, no slip base
### NS - no slip top and base
boundaryConditions = 'FS'


outputPath = f'./output/FAC-mantleConvection-{boundaryConditions}-res={res}-penaltyMethod/'

if uw.mpi.rank == 0:
    # checking if the directory demo_folder 
    # exist or not.
    if not os.path.exists(outputPath):

        # if the demo_folder directory is not present 
        # then create it.
        os.makedirs(outputPath)

# +


meshball = uw.meshing.AnnulusInternalBoundary(radiusOuter=rO, radiusInternal=rInt, radiusInner=rI, cellSize=res, cellSize_Outer=res, qdegree=3)


Processing gmsh file .meshes/uw_annulus_internalBoundary_rO2.204152249134948rInt1.9757785467128028_rI1.2041522491349481_csize0.075_csizefs0.075.msh
Mesh saved to .meshes/uw_annulus_internalBoundary_rO2.204152249134948rInt1.9757785467128028_rI1.2041522491349481_csize0.075_csizefs0.075.msh.h5


In [13]:
if uw.mpi.size == 1:
    import numpy as np
    import pyvista as pv
    import vtk

    pv.global_theme.background = "white"
    pv.global_theme.window_size = [750, 750]
    pv.global_theme.antialiasing = True
    pv.global_theme.jupyter_backend = "panel"
    pv.global_theme.smooth_shading = True
    pv.global_theme.camera["viewup"] = [0.0, 1.0, 0.0]
    pv.global_theme.camera["position"] = [0.0, 0.0, -5.0]

    meshball.vtk(outputPath+"ignore_meshball.vtk")
    pvmesh = pv.read(outputPath+"ignore_meshball.vtk")

    pl = pv.Plotter()

    # pl.add_mesh(pvmesh,'Black', 'wireframe', opacity=0.5)
    pl.add_mesh(pvmesh, cmap="coolwarm", edge_color="Black", show_edges=True, use_transparency=False, opacity=0.5)

    pl.show()

/Users/jcgraciosa/mambaforge/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2500: PyVistaDeprecationWarning: antialising is deprecated.  Please use `anti_aliasing` instead.
  warnings.warn(
/Users/jcgraciosa/mambaforge/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2468: PyVistaDeprecationWarning: `anti_aliasing` is now a string or None and must be either "ssaa", "msaa", "fxaa", or None
  warnings.warn(


/var/folders/91/xdyy27357hv1tymzym7xhf5h0000gq/T/ipykernel_65306/4205732629.py:9: PyVistaDeprecationWarning: `panel` backend is deprecated and is planned for future removal.
  pv.global_theme.jupyter_backend = "panel"


BokehModel(combine_events=True, render_bundle={'docs_json': {'b4a1eb61-4b7b-46a5-b31d-1a66245e2dd6': {'defs': …

In [14]:
v_soln = uw.discretisation.MeshVariable("U", meshball, meshball.dim, degree=2)
p_soln = uw.discretisation.MeshVariable("P", meshball, 1, degree=1)
t_soln = uw.discretisation.MeshVariable("T", meshball, 1, degree=3)
t_0    = uw.discretisation.MeshVariable("T0", meshball, 1, degree=3)

timeField    = uw.discretisation.MeshVariable("time", meshball, 1, degree=1)
density_proj = uw.discretisation.MeshVariable("rho", meshball, 1, degree=1)
visc         = uw.discretisation.MeshVariable(r"\eta", meshball, 1, degree=1, continuous=True)

SR         = uw.discretisation.MeshVariable(r"\SR", meshball, 1, degree=1, continuous=True)

meshr        = uw.discretisation.MeshVariable(r"r", meshball, 1, degree=1)

swarm = uw.swarm.Swarm(mesh=meshball)
material = uw.swarm.SwarmVariable("Mat", swarm, 1)
swarm.populate(fill_param=1)

In [15]:
# Create Stokes object
stokes = Stokes(meshball, velocityField=v_soln, pressureField=p_soln, solver_name="stokes")

### Add constitutive model
stokes.constitutive_model = uw.systems.constitutive_models.ViscousFlowModel(meshball.dim)

# Set solve options here (or remove default values
stokes.petsc_options.delValue("ksp_monitor")

In [16]:
# Create adv_diff object

adv_diff = uw.systems.AdvDiffusion(
    meshball,
    u_Field=t_soln,
    V_Field=v_soln,
    solver_name="adv_diff",
    verbose=False,
)

adv_diff.constitutive_model = uw.systems.constitutive_models.DiffusionModel(meshball.dim)

adv_diff.constitutive_model.Parameters.diffusivity = kappa

adv_diff.theta = 0.5

In [17]:
### set up swarm distribution
with swarm.access(material):
    #print(material.data.shape)
    material.data[:] = 0
    material.data[np.sqrt(swarm.data[:,0]**2 + swarm.data[:,1]**2) <= rInt] = 1

T_density = rho * (1. - (alpha * (t_soln.sym[0] - T_surf)))

# +
### create projections of vars onto mesh
nodal_rho_calc = uw.systems.Projection(meshball, density_proj)
nodal_rho_calc.uw_function = T_density
nodal_rho_calc.smoothing = 1.0e-3
nodal_rho_calc.petsc_options.delValue("ksp_monitor")

viscosity_calc = uw.systems.Projection(meshball, visc)
viscosity_calc.uw_function = stokes.constitutive_model.Parameters.viscosity
viscosity_calc.smoothing = 1.0e-3
viscosity_calc.petsc_options.delValue("ksp_monitor")

SR_calc = uw.systems.Projection(meshball, SR)
SR_calc.uw_function = stokes._Einv2
SR_calc.smoothing = 1.0e-3
SR_calc.petsc_options.delValue("ksp_monitor")

IndexError: index -4606056518893174784 is out of bounds for axis 0 with size 4706

In [ ]:
with swarm.access():
    print(material.data.shape)

(4706, 1)
